In [1]:
import cv2
from PIL import Image
import numpy as np

In [13]:
from pdf2image import convert_from_path
import re

In [95]:
pages = convert_from_path(r"..\Resources\patient_details\pd_1.pdf",poppler_path=r"D:\Naveen\OCR_Healthcare_Project\poppler-22.04.0\Library\bin")

In [57]:
pages[1].show()

In [58]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [96]:
def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5,fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,61,11)
    
    return processed_image

In [97]:
document_text = ""
for page in pages:
    process = preprocess_image(page)
    document_text += pytesseract.image_to_string(process)

In [98]:
print(document_text)

17/12/2020

Patient Medical Record

Patient Information Birth Date

Kathy Crawford May 6 1972

(737) 988-0851 Weight’

9264 Ash Dr 95

New York City, 10005 '

United States Height:
190

In Case of Emergency
ee J
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
nn ee
Chicken Pox (Varicella): Measies:
IMMUNE

IMMUNE
Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches}:

Migraine.

‘Name of Insurance Company:

Random Insuarance Company - 4789 Bollinger Rd
Jersey City, New Jersey, 07030

a Policy Number:
71 1520731 3 Expiry Date:

. 30 December 2020
Do you have medical insurance?

Yes:

Medical Insurance Details

List any allergies:

Peanuts

List any medication taken regularly:
Triptans


In [84]:
name_pattern = "Birth Date(.*?)\(\d{3}\)"
name_match = re.findall(name_pattern, document_text, flags=re.DOTALL)
name = name_match[0].strip()

In [85]:
date_pattern = '((Jan|April|Feb|March|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ \d+]*)'
date_match = re.findall(date_pattern,name)
date_match

[('May 6 1972', 'May')]

In [86]:
def remove_noise_from_name(name):
    name = name.strip()
    date_pattern = '((Jan|April|Feb|March|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[ \d+]*)'
    date_match = re.findall(date_pattern,name)
    if date_match:
        name = name.replace(date_match[0][0],"").strip()
    return name

In [87]:
remove_noise_from_name(name_match[0])

'Kathy Crawford'

In [88]:
phone_num_pattern = "Patient Information.*?(\(\d{3}\) \d{3}-\d{4})"
phone_num_match = re.findall(phone_num_pattern, document_text, flags=re.DOTALL)
phone_num_match

['(737) 988-0851']

In [89]:
hepatitsis_pattern = "Have you had the Hepatitis B vaccination\?.*?(Yes|No)"
hepatitsis_status_match = re.findall(hepatitsis_pattern,document_text,flags=re.DOTALL)
hepatitsis_status_match[0].strip()

'No'

In [99]:
medi_prob_pattern = "List any Medical Problems (\(|\{)asthma, seizures, headaches(\)|\}):(.*?)(‘Name|Name)"
medi_prob_match = re.findall(medi_prob_pattern, document_text, flags=re.DOTALL)
medi_prob_match[0][2].strip()

'Migraine.'